In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

## Bokeh imports

In [54]:
#%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
##import pandas as pd
#import numpy as np
from bokeh.models.tools import HoverTool
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

In [11]:
output_notebook()

Loading BokehJS ...

In [12]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "New_York_Times/covid-19-state-level-data.csv"

In [13]:
covid_df = pd.read_csv(covid)

In [14]:
covid_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
1824,2020-04-04,Virginia,51,2407,52
1825,2020-04-04,Washington,53,7498,318
1826,2020-04-04,West Virginia,54,282,2
1827,2020-04-04,Wisconsin,55,2112,60


## data cleaning

In [7]:
covcopy = covid_df.copy()

In [16]:
state_list = covid_df['state'].unique()

In [17]:
state_list

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands'], dtype=object)

## Isolate regions by state

In [19]:
region_list_country = covid_df.loc[covid_df['state'] == "Oregon"]

In [21]:
region_list_country.head(5)

,date,state,fips,cases,deaths
225,2020-02-28,Oregon,41,1,0
235,2020-02-29,Oregon,41,1,0
247,2020-03-01,Oregon,41,2,0
262,2020-03-02,Oregon,41,2,0
278,2020-03-03,Oregon,41,2,0


In [35]:
cases_state = [case for case in region_list_country['cases']]

In [37]:
new_case = []
for day in range(0,len(cases_state)):
    if day == 0:
        new_case.append(cases_state[day])
    elif cases_state[day] > cases_state[day - 1]:
        new_case.append(cases_state[day] - cases_state[day - 1])
    else:
        new_case.append(0)

In [38]:
region_list_country['new_cases'] = new_case

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
deaths_state = [case for case in region_list_country['deaths']]

In [41]:
new_deaths = []
for day in range(0,len(deaths_state)):
    if day == 0:
        new_deaths.append(deaths_state[day])
    elif deaths_state[day] > deaths_state[day - 1]:
        new_deaths.append(deaths_state[day] - deaths_state[day - 1])
    else:
        new_deaths.append(0)

In [42]:
region_list_country['new_deaths'] = new_deaths

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
region_list_country.head(5)

,date,state,fips,cases,deaths,new_cases,new_deaths
225,2020-02-28,Oregon,41,1,0,1,0
235,2020-02-29,Oregon,41,1,0,0,0
247,2020-03-01,Oregon,41,2,0,1,0
262,2020-03-02,Oregon,41,2,0,0,0
278,2020-03-03,Oregon,41,2,0,0,0


In [46]:
ref_date = []

for date in region_list_country['date']:
    ref_date.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

date_groups = []

for date in indy_location_moving[0]['date']:
    date_groups_1.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [49]:
date_cases = []

for date in ref_date:
    date_cases.append(str(date))

# pass plot data as well as hovertool data in to plot.
source_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['cases'],
            desc=date_cases,
        )
    )

In [58]:
# pass plot data as well as hovertool data in to plot.
source_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['deaths'],
            desc=date_cases,
        )
    )

In [52]:
# configure plot tools including the hover tool / tooltips.  Displays date / time of individual data points when hovering 
#over individual points.
TOOLSCCD = [HoverTool(tooltips=[
            ("cvd_date", "@desc"),("cases","$y")],
                  formatters={'DateTime': 'datetime'}),'wheel_zoom','save','crosshair','pan','reset']

In [56]:
cid = figure(title = f"Values for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cid.xaxis.axis_label = 'Event Date'
cid.yaxis.axis_label = 'COVID Total Cases'
# Setup x axis for using Datetime formatting.
cid.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cid.xaxis.major_label_orientation = pi/4
cid.circle('x', 'y', line_color="navy", fill_color="red", fill_alpha=0.2, size=10,source=source_cases)


GlyphRenderer(id='1110', ...)

In [59]:
cix = figure(title = f"Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cix.xaxis.axis_label = 'Event Date'
cix.yaxis.axis_label = 'COVID Total Deaths'
# Setup x axis for using Datetime formatting.
cix.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cix.xaxis.major_label_orientation = pi/4
cix.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_deaths)


GlyphRenderer(id='1279', ...)

In [60]:
show(cid)

In [61]:
show(cix)